In [1]:
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

In [2]:
# Initialize the Dash app with callback exception suppression
app = JupyterDash(__name__, suppress_callback_exceptions=True)

# Load the dataset
df = pd.read_csv('worldometer_data.csv')

/home/aol8/miniconda3/lib/python3.11/site-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [3]:
# World map visualization of total cases
fig_map = px.scatter_geo(
    df,
    locations="Country/Region",
    locationmode="country names",
    color="TotalCases",
    size="TotalCases",
    hover_name="Country/Region",
    size_max=50,
    template="plotly_dark",
    projection="natural earth",
    title="Total COVID-19 Cases Worldwide",
    color_continuous_scale=px.colors.sequential.Plasma
)

In [4]:
# Create summary cards
def summary_card(title, value, color):
    return html.Div([
        html.H4(title, style={'color': '#FFFFFF', 'margin': '0'}),
        html.P(value, style={'color': color, 'fontSize': '24px', 'margin': '0'}),
    ], style={
        'padding': '20px',
        'backgroundColor': '#2B2B2B',
        'borderRadius': '10px',
        'margin': '10px',
        'textAlign': 'center',
        'flex': '1',
        'maxWidth': '200px'
    })

# Sidebar layout
sidebar = html.Div([
    html.H2("COVID-19 Dashboard", style={'color': '#FFFFFF', 'textAlign': 'center'}),
    html.Hr(),
    html.P("Navigation", style={'color': '#FFFFFF'}),
    dcc.Link('Overview', href='/overview', style={'color': '#FFFFFF', 'display': 'block', 'padding': '10px'}),
    dcc.Link('Statistics', href='/statistics', style={'color': '#FFFFFF', 'display': 'block', 'padding': '10px'}),
    dcc.Link('Country Comparisons', href='/comparisons', style={'color': '#FFFFFF', 'display': 'block', 'padding': '10px'}),
    dcc.Link('About', href='/about', style={'color': '#FFFFFF', 'display': 'block', 'padding': '10px'}),
], style={
    'backgroundColor': '#2B2B2B',
    'padding': '20px',
    'width': '300px',
    'height': '100vh',
    'position': 'fixed',
    'display': 'flex',
    'flexDirection': 'column'
})

In [5]:
# Define page content layouts
def render_overview():
    return html.Div([
        html.H1("COVID-19 Global Overview", style={'textAlign': 'center', 'color': '#FFFFFF'}),
        html.Div([
            summary_card("Total Cases", f"{df['TotalCases'].sum():,}", '#FF5733'),
            summary_card("Total Deaths", f"{df['TotalDeaths'].sum():,}", '#C70039'),
            summary_card("Total Recovered", f"{df['TotalRecovered'].sum():,}", '#28B463'),
            summary_card("Total Active Cases", f"{df['ActiveCases'].sum():,}", '#FFC300'),
        ], style={
            'display': 'flex',
            'justifyContent': 'space-between',
            'marginTop': '20px',
            'flexWrap': 'wrap'
        }),
        html.Hr(),
        dcc.Graph(figure=fig_map),
    ], style={
        'padding': '20px',
        'marginLeft': '300px',
        'backgroundColor': '#121212',
        'color': '#FFFFFF',
        'minHeight': '100vh'
    })


In [6]:
def render_statistics():
    # Bar Chart: Top 10 countries with the highest total cases
    top_countries = df.nlargest(10, 'TotalCases')
    bar_chart = px.bar(
        top_countries,
        x='Country/Region',
        y='TotalCases',
        color='TotalCases',
        title='Top 10 Countries with the Highest Total Cases',
        template='plotly_dark',
        labels={'TotalCases': 'Total Cases'},
        color_continuous_scale=px.colors.sequential.Plasma
    )

    # Pie Chart: Global distribution of cases
    pie_chart = px.pie(
        df,
        names='Country/Region',
        values='TotalCases',
        title='Global Distribution of COVID-19 Cases',
        template='plotly_dark'
    )

    return html.Div([
        html.H1("Global and Regional Statistics", style={'textAlign': 'center', 'color': '#FFFFFF'}),
        dcc.Graph(figure=bar_chart),
        html.Hr(),
        dcc.Graph(figure=pie_chart)
    ], style={
        'padding': '20px',
        'marginLeft': '300px',
        'backgroundColor': '#121212',
        'color': '#FFFFFF'
    })


In [7]:
def render_comparisons():
    # Dropdown for country selection with readable colors
    dropdown = dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in df['Country/Region'].unique()],
        value=['India'],  # Default selected country
        multi=True,
        placeholder="Select countries...",
        style={
            'backgroundColor': '#FFFFFF',  # White background for dropdown
            'color': '#000000',           # Black text for dropdown options
            'border': '1px solid #555',   # Optional border for better visibility
            'padding': '5px',             # Adjust padding for better spacing
        }
    )

    return html.Div([
        html.H1("Country Comparisons", style={'textAlign': 'center', 'color': '#FFFFFF'}),
        html.Div([
            html.Label("Select Countries for Comparison:", style={'color': '#FFFFFF'}),
            dropdown
        ], style={'padding': '10px'}),
        html.Div(id='comparison-output')  # Placeholder for comparison graphs
    ], style={
        'padding': '20px',
        'marginLeft': '300px',
        'backgroundColor': '#121212',
        'color': '#FFFFFF'
    })

In [8]:
def render_about():
    return html.Div([
        html.H1("About This Dashboard", style={'textAlign': 'center', 'color': '#FFFFFF'}),
        html.P("This dashboard provides insights on COVID-19 data around the world. The dataset was gotten from Kaggle.", style={'color': '#FFFFFF'}),
    ], style={'padding': '20px', 'marginLeft': '300px', 'backgroundColor': '#121212', 'color': '#FFFFFF'})

# Define the app layout
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div([
        sidebar,
        html.Div(id='page-content', style={
            'flex': '1',
            'padding': '20px',
            'backgroundColor': '#121212',
            'color': '#FFFFFF'
        })
    ], style={
        'display': 'flex',
        'width': '100vw',
        'height': '100vh'
    })
])

In [9]:
# Callback to update content based on URL
@app.callback(
    Output('page-content', 'children'),
    Input('url', 'pathname')
)
def display_page(pathname):
    if pathname == '/overview' or pathname == '/':
        return render_overview()
    elif pathname == '/statistics':
        return render_statistics()
    elif pathname == '/comparisons':
        return render_comparisons()
    elif pathname == '/about':
        return render_about()
    else:
        return html.H1("404: Page Not Found", style={'textAlign': 'center', 'color': '#FFFFFF'})


In [10]:
# Callback for dynamic country comparison
@app.callback(
    Output('comparison-output', 'children'),
    Input('country-dropdown', 'value')
)
def update_comparison(selected_countries):
    if not selected_countries:
        return html.P("Select at least one country.", style={'color': '#FFFFFF'})

    filtered_df = df[df['Country/Region'].isin(selected_countries)]

    # Bar Chart for comparison
    comparison_chart = px.bar(
        filtered_df,
        x='Country/Region',
        y=['TotalCases', 'TotalDeaths', 'TotalRecovered'],
        title='Country Comparison of Key Metrics',
        template='plotly_dark',
        labels={'value': 'Count', 'variable': 'Metric'},
        barmode='group'
    )

    return dcc.Graph(figure=comparison_chart)


In [11]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8058)

Dash app running on http://127.0.0.1:8058/
